In [1]:
import torch
import numpy as np
from torch.autograd import grad
from torch.autograd.functional import hessian, jacobian

In [2]:
var = torch.tensor([[5.0, 2.0, 3.0],
                   [8.0, 4.0, 4.0]], requires_grad=True)
t = torch.tensor([[5.0]], requires_grad=True)
u = lambda x, t: torch.exp(x[:, 0]) + (1/6)*torch.pow(x[:, 1], 3)

In [3]:
out = u(var, t)
div = torch.sum(grad(outputs=out, inputs=var, grad_outputs=torch.ones_like(out))[0], dim=-1)
div

tensor([ 150.4132, 2988.9580])

In [4]:
out = u(var, t)
out

tensor([ 149.7465, 2991.6248], grad_fn=<AddBackward0>)

In [5]:
def Δ(u, vars):
    u_gradient = grad(outputs=u, inputs=vars, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_laplacian = torch.sum([grad(u_gradient_i, vars_i, create_graph=True) for u_gradient_i, vars_i in zip(vars, u_gradient)])
    return u_laplacian

In [6]:
'''u_gradient = grad(outputs=out, inputs=var, grad_outputs=torch.ones_like(out), create_graph=True)[0]
u_gradient = u_gradient.flatten(start_dim=1)
var = var.flatten()
[grad(u_gradient_i, vars_i, create_graph=True)[0] for u_gradient_i, vars_i in zip(var[:], (u_gradient[..., i] for i in range(len(var))))]'''

'u_gradient = grad(outputs=out, inputs=var, grad_outputs=torch.ones_like(out), create_graph=True)[0]\nu_gradient = u_gradient.flatten(start_dim=1)\nvar = var.flatten()\n[grad(u_gradient_i, vars_i, create_graph=True)[0] for u_gradient_i, vars_i in zip(var[:], (u_gradient[..., i] for i in range(len(var))))]'

In [7]:
def laplace(fx: torch.Tensor, x: torch.Tensor):
    dfx = torch.autograd.grad(fx, x, create_graph=True, grad_outputs=torch.ones_like(fx))[0]
    vec = torch.ones_like(dfx)
    ddfx = torch.autograd.grad(dfx, x, grad_outputs=vec)[0]
    return ddfx

In [8]:
from operators import Δ
Δ(out.unsqueeze(1), var)

torch.Size([2, 1]) torch.Size([2, 3])
torch.Size([2, 3])
torch.Size([2, 3])
torch.Size([2, 1])


tensor([[ 150.4132],
        [2984.9580]], grad_fn=<UnsqueezeBackward0>)